In [ ]:
#| include: false
! [ -e /content ] && pip install -Uqq gingado nbdev # install or upgrade gingado on colab

In [ ]:
#| default_exp benchmark
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
#| echo: false

# Code below included to ensure compatibility with scikit-learn v1.1.x
from sklearn import set_config
set_config(display='text')

In [ ]:
#| include: false
from nbdev.showdoc import show_doc

A Benchmark object has a similar API to a `sciki-learn` estimator: you build an instance with the desired arguments, and fit it to the data at a later moment. Benchmarks is a convenience wrapper for reading the training data, passing it through a simplified pipeline consisting of data imputation and a standard scalar, and then the benchmark function calibrated with a grid search.

A `gingado` Benchmark object seeks to automatise a significant part of creating a benchmark model. Importantly, the Benchmark object also has a `compare` method that helps users evaluate if candidate models are better than the benchmark, and if one of them is, it becomes the new benchmark. This `compare` method takes as argument another fitted estimator (which could be itself a solo estimator or a whole pipeline) or a list of fitted estimators. 

Benchmarks start with default values that should perform reasonably well in most settings, but the user is also free to choose any of the benchmark's components by passing as arguments the data split, pipeline, and/or a dictionary of parameters for the hyperparameter tuning.

## Base class

`gingado` has a `ggdBenchmark` base class that contains the basic functionalities for Benchmark objects. It is not meant to be used by itself, but only as a hyperclass for Benchmark objects. `gingado` ships with two of these objects that subclass `ggdBenchmark`: `ClassificationBenchmark` and `RegressionBenchmark`. They are both described below in their respective sections.

Users are encouraged to submit a PR with their own benchmark models subclassing `ggdBenchmark`.

In [ ]:
#| include: false
#| export
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.model_selection import TimeSeriesSplit, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.utils.metaestimators import available_if
from sklearn.utils.validation import check_is_fitted
from gingado.model_documentation import ggdModelDocumentation, ModelCard

def _benchmark_has(attr):
    # Check if the benchmark has certain attributes
    def check(self):
        getattr(self.benchmark, attr)
        return True
    return check
        
class ggdBenchmark(BaseEstimator):
    "The base class for gingado's Benchmark objects."
    
    def _check_is_time_series(self, X, y=None):
        """
        Checks whether the data is a time series, and sets a data splitter
        accordingly if no data splitter is provided by the user
        Note: all data without an index (eg, a Numpy array) are considered to NOT be a time series
        """
        if hasattr(X, "index"):
            self.is_timeseries = pd.core.dtypes.common.is_datetime_or_timedelta_dtype(X.index)
        else:
            self.is_timeseries = False
        if self.is_timeseries and y is not None:
            if hasattr(y, "index"):
                self.is_timeseries = pd.core.dtypes.common.is_datetime_or_timedelta_dtype(y.index)
            else:
                self.is_timeseries = False

        if self.cv is None:
            self.cv = TimeSeriesSplit() if self.is_timeseries else StratifiedShuffleSplit()

    def _creates_estimator(self):
        if self.estimator is None:
            pass

    def _fit(self, X, y):
        self._check_is_time_series(X, y)

        X, y = self._validate_data(X, y)

        if hasattr(self.estimator, "random_state"):
            self.estimator.random_state = self.random_state

        if self.param_search and self.param_grid:                
            self.benchmark = self.param_search(estimator=self.estimator, param_grid=self.param_grid, scoring=self.scoring, verbose=self.verbose_grid)
        else:
            self.benchmark = self.estimator
            
        self.benchmark.fit(X, y)

        if self.auto_document is not None:
            self.document()

        return self
    
    def set_benchmark(
        self,
        estimator # A fitted estimator object
        ):
        # Define a fitted `estimator` as the new benchmark model
        check_is_fitted(estimator)
        self.benchmark = estimator

    def _read_candidate_params(self, candidates, ensemble_method):
                param_grid = []
                for i, model in enumerate(candidates):
                    check_is_fitted(model)
                    param_grid.append({
                        **{'candidate_estimator': [model]},
                        **{
                            'candidate_estimator__' + k: (v,)
                            for k, v in model.get_params().items()
                        }}
                    )
                if ensemble_method is not None:
                    candidate_models = [('candidate_'+str(i+1), model) for i, model in enumerate(candidates)]
                    voting = ensemble_method(estimators=candidate_models)
                    ensemble = {'candidate_estimator': [voting]}
                    param_grid.append(ensemble)
                return param_grid

    def compare(
        self,
        X:np.ndarray, # Array-like data of shape (n_samples, n_features)
        y:np.ndarray, # Array-like data of shape (n_samples,) or (n_samples, n_targets)
        candidates, # Candidate estimator or list of candidate estimator(s)
        ensemble_method='object_default', 
        update_benchmark:bool=True # Whether to use the best performing candidate model as the new benchmark
        ):
        "Use a testing dataset to compare the performance of the fitted benchmark model with one or more candidate models using a grid search"
    
        check_is_fitted(self.benchmark)
        old_benchmark_params = self.benchmark.get_params()

        candidates = list(candidates) if type(candidates) != list else candidates
        list_candidates = [self.benchmark] + candidates
        
        est = self.benchmark.best_estimator_ if hasattr(self.benchmark, "best_estimator_") else self.benchmark
        cand_pipeline = Pipeline([('candidate_estimator', est)])
        
        if ensemble_method == 'object_default':
            ensemble_method = self.ensemble_method
        cand_params = self._read_candidate_params(list_candidates, ensemble_method=ensemble_method)
        cand_grid = GridSearchCV(cand_pipeline, cand_params, verbose=self.verbose_grid).fit(X, y)
        
        self.model_comparison_ = cand_grid

        if update_benchmark:
            if cand_grid.best_estimator_.get_params() != old_benchmark_params:
                self.set_benchmark(cand_grid)
                print("Benchmark updated!")
                print("New benchmark:")
                print(self.benchmark.best_estimator_)

        if self.auto_document is not None:
            self.document()

    def compare_fitted_candidates(self, X, y, candidates, scoring_func):
        check_is_fitted(self.benchmark)
        candidates = list(candidates) if type(candidates) != list else candidates
        for candidate in candidates:
            check_is_fitted(candidate)
        list_candidates = [self.benchmark] + candidates
        
        return {candidate.__repr__(): scoring_func(y, candidate.predict(X)) for candidate in list_candidates}

    def _read_attr(self):
        for a in dir(self.benchmark):
            if a == '_estimator_type' or a.endswith("_") and not a.startswith("_") and not a.endswith("__"):
                try:
                    model_attr = self.benchmark.__getattribute__(a)
                    yield {a: model_attr}
                except:
                    pass

    def document(
        self, 
        documenter=None # A gingado Documenter or the documenter set in `auto_document` if None.
        ):
        "Document the benchmark model using the template in `documenter`"
        documenter = self.auto_document if documenter is None else documenter
        self.model_documentation = documenter()
        model_info = list(self._read_attr())
        model_info = {k:v for i in model_info for k, v in i.items()}
        #self.model_documentation.read_model(self.benchmark)
        self.model_documentation.fill_model_info(model_info)

    @available_if(_benchmark_has("predict"))
    def predict(self, X, **predict_params):
        return self.benchmark.predict(X, **predict_params)

    @available_if(_benchmark_has("fit_predict"))
    def fit_predict(self, X, y=None, **predict_params):
        return self.benchmark.fit_predict(X, y, **predict_params)

    @available_if(_benchmark_has("predict_proba"))
    def predict_proba(self, X, **predict_proba_params):
        return self.benchmark.predict_proba(X, **predict_proba_params)

    @available_if(_benchmark_has("decision_function"))
    def decision_function(self, X):
        return self.benchmark.decision_function(X)

    @available_if(_benchmark_has("score"))
    def score(self, X):
        return self.benchmark.score(X)

    @available_if(_benchmark_has("score_samples"))
    def score_samples(self, X):
        return self.benchmark.score_samples(X)

    @available_if(_benchmark_has("predict_log_proba"))
    def predict_log_proba(self, X, **predict_log_proba_params):
        return self.benchmark.predict_log_proba(X, **predict_log_proba_params)

In [ ]:
show_doc(ggdBenchmark)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L24){target="_blank" style="float:right; font-size:smaller"}

### ggdBenchmark

>      ggdBenchmark ()

The base class for gingado's Benchmark objects.

In [ ]:
show_doc(ggdBenchmark.compare)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L96){target="_blank" style="float:right; font-size:smaller"}

### ggdBenchmark.compare

>      ggdBenchmark.compare (X:numpy.ndarray, y:numpy.ndarray, candidates,
>                            ensemble_method='object_default',
>                            update_benchmark:bool=True)

Use a testing dataset to compare the performance of the fitted benchmark model with one or more candidate models using a grid search

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | ndarray |  | Array-like data of shape (n_samples, n_features) |
| y | ndarray |  | Array-like data of shape (n_samples,) or (n_samples, n_targets) |
| candidates |  |  | Candidate estimator or list of candidate estimator(s) |
| ensemble_method | str | object_default |  |
| update_benchmark | bool | True | Whether to use the best performing candidate model as the new benchmark |

In [ ]:
show_doc(ggdBenchmark.document)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L150){target="_blank" style="float:right; font-size:smaller"}

### ggdBenchmark.document

>      ggdBenchmark.document (documenter=None)

Document the benchmark model using the template in `documenter`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| documenter | NoneType | None | A gingado Documenter or the documenter set in `auto_document` if None. |

## Classification tasks

The default benchmark for classification tasks is a [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) object. Its parameters are fine-tuned in each case according to the user's data.

In [ ]:
#| include: false
#| export
import numpy as np
from gingado.model_documentation import ModelCard
from sklearn.base import ClassifierMixin
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

class ClassificationBenchmark(ggdBenchmark, ClassifierMixin):
    "A gingado Benchmark object used for classification tasks"
    def __init__(self, 
    cv=None, 
    estimator=RandomForestClassifier(oob_score=True), 
    param_grid={'n_estimators': [100, 250], 'max_features': ['sqrt', 'log2', None]}, 
    param_search=GridSearchCV, 
    scoring=None, 
    auto_document=ModelCard, 
    random_state=None,
    verbose_grid=False,
    ensemble_method=VotingClassifier):
        self.cv = cv
        self.estimator = estimator
        self.param_grid = param_grid
        self.param_search = param_search
        self.scoring = scoring
        self.auto_document = auto_document
        self.random_state = random_state
        self.verbose_grid = verbose_grid
        self.ensemble_method = ensemble_method

    def fit(
        self, 
        X:np.ndarray, # Array-like data of shape (n_samples, n_features)
        y=None # Array-like data of shape (n_samples,) or (n_samples, n_targets) or None
        ):
        "Fit the `ClassificationBenchmark` model"
        self._fit(X, y)
        return self

In [ ]:
show_doc(ClassificationBenchmark)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L198){target="_blank" style="float:right; font-size:smaller"}

### ClassificationBenchmark

>      ClassificationBenchmark (cv=None,
>                               estimator=RandomForestClassifier(oob_score=True)
>                               , param_grid={'n_estimators': [100, 250],
>                               'max_features': ['sqrt', 'log2', None]},
>                               param_search=<class
>                               'sklearn.model_selection._search.GridSearchCV'>,
>                               scoring=None, auto_document=<class
>                               'gingado.model_documentation.ModelCard'>,
>                               random_state=None, verbose_grid=False,
>                               ensemble_method=<class
>                               'sklearn.ensemble._voting.VotingClassifier'>)

A gingado Benchmark object used for classification tasks

In [ ]:
show_doc(ClassificationBenchmark.fit)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L220){target="_blank" style="float:right; font-size:smaller"}

### ClassificationBenchmark.fit

>      ClassificationBenchmark.fit (X:numpy.ndarray, y=None)

Fit the `ClassificationBenchmark` model

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | ndarray |  | Array-like data of shape (n_samples, n_features) |
| y | NoneType | None | Array-like data of shape (n_samples,) or (n_samples, n_targets) or None |

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
# some mock up data
X, y = make_classification()

# the gingado benchmark
bm = ClassificationBenchmark(verbose_grid=3).fit(X, y)

# note that now the `bm` object can be used as an estimator
assert bm.predict(X).shape == y.shape

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_features=sqrt, n_estimators=100;, score=1.000 total time=   0.1s
[CV 2/5] END max_features=sqrt, n_estimators=100;, score=0.750 total time=   0.1s
[CV 3/5] END max_features=sqrt, n_estimators=100;, score=0.950 total time=   0.1s
[CV 4/5] END max_features=sqrt, n_estimators=100;, score=1.000 total time=   0.1s
[CV 5/5] END max_features=sqrt, n_estimators=100;, score=0.950 total time=   0.1s
[CV 1/5] END max_features=sqrt, n_estimators=250;, score=1.000 total time=   0.3s
[CV 2/5] END max_features=sqrt, n_estimators=250;, score=0.750 total time=   0.3s
[CV 3/5] END max_features=sqrt, n_estimators=250;, score=0.950 total time=   0.3s
[CV 4/5] END max_features=sqrt, n_estimators=250;, score=1.000 total time=   0.3s
[CV 5/5] END max_features=sqrt, n_estimators=250;, score=0.900 total time=   0.3s
[CV 1/5] END max_features=log2, n_estimators=100;, score=1.000 total time=   0.1s
[CV 2/5] END max_features=log2, n_esti

Importantly, `gingado` automatically provides some information to help the user documentat the benchmark model. More specifically, `ggdBenchmark` objects collect model information and pass it to a dictionary with key `info` in a field called `model_details`. 

In [ ]:
bm.model_documentation.show_json()

{'model_details': {'developer': 'Person or organisation developing the model',
  'datetime': '2022-09-24 00:37:21 ',
  'version': 'Model version',
  'type': 'Model type',
  'info': {'_estimator_type': 'classifier',
   'best_estimator_': RandomForestClassifier(max_features=None, oob_score=True),
   'best_index_': 4,
   'best_params_': {'max_features': None, 'n_estimators': 100},
   'best_score_': 0.95,
   'classes_': array([0, 1]),
   'cv_results_': {'mean_fit_time': array([0.11072736, 0.30638943, 0.12044678, 0.29842086, 0.11654415,
           0.31644664]),
    'std_fit_time': array([0.00281184, 0.01535436, 0.0065928 , 0.01652688, 0.00285051,
           0.00959558]),
    'mean_score_time': array([0.00769496, 0.0195919 , 0.00786419, 0.0195241 , 0.00688624,
           0.0202014 ]),
    'std_score_time': array([0.00060471, 0.00127096, 0.0007323 , 0.00172929, 0.00021744,
           0.00237004]),
    'param_max_features': masked_array(data=['sqrt', 'sqrt', 'log2', 'log2', None, None],
      

It is also simple to define as benchmark a model that you already fitted and still benefit from the other functionalities provided by `Benchmark` class. This can also be done in case you are using a saved version of a fitted model (eg, the model you are using in production) and want to have that as the benchmark.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier().fit(X, y)

bm.set_benchmark(estimator=forest)

assert forest == bm.benchmark
assert hasattr(bm.benchmark, "predict")
assert bm.predict(X).shape == y.shape

## Regression tasks

The default benchmark for regression tasks is a [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) object.  Its parameters are fine-tuned in each case according to the user's data.

In [ ]:
#| include: false
#| export
import numpy as np
from gingado.model_documentation import ModelCard
from sklearn.base import RegressorMixin
from sklearn.ensemble import RandomForestRegressor, VotingRegressor

class RegressionBenchmark(ggdBenchmark, RegressorMixin):
    "A gingado Benchmark object used for regression tasks"
    def __init__(self, 
    cv=None, 
    estimator=RandomForestRegressor(oob_score=True), 
    param_grid={'n_estimators': [100, 250], 'max_features': ['sqrt', 'log2', None]}, 
    param_search=GridSearchCV, 
    scoring=None, 
    auto_document=ModelCard, 
    random_state=None,
    verbose_grid=False,
    ensemble_method=VotingRegressor):
        self.cv = cv
        self.estimator = estimator
        self.param_grid = param_grid
        self.param_search = param_search
        self.scoring = scoring
        self.auto_document = auto_document
        self.random_state = random_state
        self.verbose_grid = verbose_grid
        self.ensemble_method = ensemble_method

    def fit(
        self, 
        X:np.ndarray, # Array-like data of shape (n_samples, n_features)
        y=None # Array-like data of shape (n_samples,) or (n_samples, n_targets) or None
        ):
        "Fit the `RegressionBenchmark` model"
        self._fit(X, y)
        return self

In [ ]:
show_doc(RegressionBenchmark)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L236){target="_blank" style="float:right; font-size:smaller"}

### RegressionBenchmark

>      RegressionBenchmark (cv=None,
>                           estimator=RandomForestRegressor(oob_score=True),
>                           param_grid={'n_estimators': [100, 250],
>                           'max_features': ['sqrt', 'log2', None]},
>                           param_search=<class
>                           'sklearn.model_selection._search.GridSearchCV'>,
>                           scoring=None, auto_document=<class
>                           'gingado.model_documentation.ModelCard'>,
>                           random_state=None, verbose_grid=False,
>                           ensemble_method=<class
>                           'sklearn.ensemble._voting.VotingRegressor'>)

A gingado Benchmark object used for regression tasks

In [ ]:
show_doc(RegressionBenchmark.fit)

---

[source](https://github.com/dkgaraujo/gingado/blob/main/gingado/benchmark.py#L258){target="_blank" style="float:right; font-size:smaller"}

### RegressionBenchmark.fit

>      RegressionBenchmark.fit (X:numpy.ndarray, y=None)

Fit the `RegressionBenchmark` model

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | ndarray |  | Array-like data of shape (n_samples, n_features) |
| y | NoneType | None | Array-like data of shape (n_samples,) or (n_samples, n_targets) or None |

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
# some mock up data
X, y = make_regression()

# the gingado benchmark
bm = RegressionBenchmark(verbose_grid=2).fit(X, y)

# note that now the `bm` object can be used as an estimator
assert bm.predict(X).shape == y.shape

# the user might also like to set another model as the benchmark
adaboost = AdaBoostRegressor().fit(X, y)
bm.set_benchmark(estimator=adaboost)

assert adaboost == bm.benchmark
assert hasattr(bm.benchmark, "predict")
assert bm.predict(X).shape == y.shape

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ................max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ................max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ................max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ................max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ................max_features=sqrt, n_estimators=100; total time=   0.1s
[CV] END ................max_features=sqrt, n_estimators=250; total time=   0.3s
[CV] END ................max_features=sqrt, n_estimators=250; total time=   0.3s
[CV] END ................max_features=sqrt, n_estimators=250; total time=   0.3s
[CV] END ................max_features=sqrt, n_estimators=250; total time=   0.3s
[CV] END ................max_features=sqrt, n_estimators=250; total time=   0.3s
[CV] END ................max_features=log2, n_estimators=100; total time=   0.1s
[CV] END ................max_features=log2, n_est

Below we compare the benchmark (set above manually to be the adaboost algorithm) with two other candidate models: a Gaussian process and a linear Support Vector Machine (SVM).

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import LinearSVR

In [ ]:
gauss_reg = GaussianProcessRegressor().fit(X, y)
svm_reg = LinearSVR().fit(X, y)

bm.compare(X, y, candidates=[gauss_reg, svm_reg])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END candidate_estimator=AdaBoostRegressor(), candidate_estimator__base_estimator=None, candidate_estimator__learning_rate=1.0, candidate_estimator__loss=linear, candidate_estimator__n_estimators=50, candidate_estimator__random_state=None; total time=   0.1s
[CV] END candidate_estimator=AdaBoostRegressor(), candidate_estimator__base_estimator=None, candidate_estimator__learning_rate=1.0, candidate_estimator__loss=linear, candidate_estimator__n_estimators=50, candidate_estimator__random_state=None; total time=   0.1s
[CV] END candidate_estimator=AdaBoostRegressor(), candidate_estimator__base_estimator=None, candidate_estimator__learning_rate=1.0, candidate_estimator__loss=linear, candidate_estimator__n_estimators=50, candidate_estimator__random_state=None; total time=   0.1s
[CV] END candidate_estimator=AdaBoostRegressor(), candidate_estimator__base_estimator=None, candidate_estimator__learning_rate=1.0, candidate_estimator

Note that when the benchmark object finds a model that performs better than it does, the user is informed that the benchmark is updated and the new benchmark model is shown. This only happens when the argument `update_benchmark` is set to True (as default).

Below we can see by how much it outperformed the other candidates, including the previous benchmark model and an ensemble of the previous benchmark and all the candidates. It is also a good opportunity to see how stable the performance of each model was, as judged by the standard deviation of the scores across the validation folds.

In [ ]:
pd.DataFrame(bm.benchmark.cv_results_)[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]

,params,mean_test_score,std_test_score,rank_test_score
0,{'candidate_estimator': (DecisionTreeRegressor...,0.354251,0.092104,1
1,{'candidate_estimator': GaussianProcessRegress...,-0.138693,0.137167,4
2,"{'candidate_estimator': LinearSVR(), 'candidat...",0.298425,0.103647,2
3,{'candidate_estimator': VotingRegressor(estima...,0.216893,0.097473,3


## General comments on benchmarks

### Scoring

`ClassificationBenchmark` and `RegressionBenchmark` both use the default scoring method for comparing model alternatives, both during estimation of the benchmark model and when comparing this benchmark with candidate models. Users are encouraged to consider if another scoring method is more suitable for their use case. More information on available scoring methods that are compatible with `gingado` Benchmark objects can be found [here](https://scikit-learn.org/stable/modules/model_evaluation.html).

### Data split

`gingado` benchmarks rely on hyperparameter tuning to discover the benchmark specification that is most likely to perform better with the user data. This tuning in turn depends on a data splitting strategy for the cross-validation. By default, `gingado` uses `StratifiedShuffleSplit` if the data is not time series and `TimeSeriesSplit` otherwise.

Please refer to [this page](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) for more information on the different `Splitter` classes available on `scikit-learn`, and [this page](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py) for practical advice on how to choose a splitter for data that are not time series. Any one of these objects (or a custom splitter that is compatible with them) can be passed to a `Benchmark` object.

The API does not accept custom parameters for the splitters. Users that wish to use specific parameters should include the actual `Splitter` object as the parameter.

### Custom benchmarks

`gingado` provides users with two `Benchmark` objects out of the box: `ClassificationBenchmark` and `RegressionBenchmark`, to be used depending on the task at hand. Both classes derive from a base class `ggdBenchmark`, which implements methods that facilitate model comparison. Users that want to create a customised benchmark model for themselves have two options:

* the simpler possibility is to train the estimator as usual, and then assign the fitted estimator to a `Benchmark` object. 

* if the user wants more control over the fitting process of estimating the benchmark, they can create a class that subclasses from `ggdBenchmark` and either implements custom `fit`, `predict` and `score` methods, or also subclasses from [`scikit-learn`'s `BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html). 
  * In any case, if the user wants the benchmark to automatically detect if the data is a time series and also to document the model right after fitting, the `fit` method should call `self._fit` on the data. Otherwise, the user can simply implement any consistent logic in fit as the user sees fit (pun intended).


In [ ]:
#| echo: false
import nbdev; nbdev.nbdev_export()